In [1]:
import os
import csv
import random
import pydicom
import numpy as np
import pandas as pd
from skimage import io
from skimage import measure
from skimage.transform import resize

import tensorflow as tf
from tensorflow import keras

from matplotlib import pyplot as plt
import matplotlib.patches as patches

In [2]:
# Keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense

Using TensorFlow backend.


In [3]:
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler

In [4]:
df = pd.read_csv('all/stage_1_detailed_class_info.csv')
test_Layer = pd.read_csv('all/stage_1_sample_submission.csv')
targets_df = pd.read_csv('all/stage_1_train_labels.csv')

In [5]:
train_images = []
for i in df.patientId.unique():
    train_images.append('all/train_images/%s.dcm' % i)

In [6]:
count = 100
ds = []
for i in range(100):
    ds.append(np.array(pydicom.dcmread(train_images[i]).pixel_array))

In [7]:
ds=np.array(ds)

In [8]:
ds.shape

(100, 1024, 1024)

In [17]:
#ds = pydicom.dcmread(train_images[0])

In [18]:
#ds.dir('pat')

In [19]:
#np.savetxt("pixel_array.csv", pixel_array, delimiter=",")

In [20]:
#plt.imshow(ds.pixel_array,cmap=plt.cm.bone)

In [27]:
ds.shape[1]*ds.shape[2]

1048576

In [28]:
# We want to flatten our image of 28x28 pixels to a 1D array of 784 pixels
X_train = ds.reshape(ds.shape[0], 1048576)

print("Training Shape:", X_train.shape)


Training Shape: (100, 1048576)


In [29]:
# Next, we normalize our training data to be between 0 and 1
scaler = MinMaxScaler().fit(X_train)

X_train = scaler.transform(X_train)


C:\Users\James\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [30]:
X_train

array([[0.56470588, 0.50196078, 0.44705882, ..., 0.09745763, 0.0766129 ,
        0.02690583],
       [0.12156863, 0.09411765, 0.0745098 , ..., 0.64830508, 0.61693548,
        0.67713004],
       [0.01568627, 0.04705882, 0.04313725, ..., 0.26271186, 0.33870968,
        0.19282511],
       ...,
       [0.38431373, 0.35294118, 0.3254902 , ..., 0.18220339, 0.16935484,
        0.17488789],
       [0.03137255, 0.02745098, 0.02352941, ..., 0.17372881, 0.16129032,
        0.10313901],
       [0.00392157, 0.00392157, 0.00784314, ..., 0.06355932, 0.06854839,
        0.02690583]])

In [11]:
num_classes = 2

In [12]:
y_train = targets_df[:100].Target.values

In [13]:
y_train = to_categorical(y_train, num_classes)

In [ ]:
# Create an empty sequential model
model = Sequential()

# Add the first layer where the input dimensions are the 784 pixel values
# We can also choose our activation function. `relu` is a common
model.add(Dense(256, activation='relu', input_dim=X_train.shape[1]))

#Add a second hidden layer
model.add(Dense(512, activation='relu'))

# Add our final output layer where the number of nodes 
# corresponds to the number of y labels
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss="categorical_crossentropy", # sparse_categorical_crossentropy
              optimizer="adam", metrics=['accuracy'])


# Fit (train) the model
model.fit(
    X_train,
    y_train,
    epochs=10,
    shuffle=True,
    verbose=2,
    validation_split = .2
)